In [84]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import numdifftools as nd
from tqdm.auto import tqdm
from sklearn.linear_model import LinearRegression, Ridge

In [85]:
#ead training and test data sets into data fame tormat
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [86]:
X = df_train.to_numpy()[:,2:]
X_test = df_test.to_numpy()[:,1:]
y = df_train.to_numpy()[:,1]
#w0 = [1,2,3,4,5,6,7,8,9]

## closed form solution

In [93]:
#first calculate the closed form solution
X_T = X.T
w_closed = (np.linalg.inv(X_T @ X) @ X_T) @ y
y_closed = X @ w_closed

#computes the root mean square error
RMSE = mean_squared_error(y, y_closed)**0.5
print(RMSE)

#np.sqrt(np.mean((y-y_closed)**2))

2.4661713660845003e-13


In [94]:
# strore return format as Id, y in a data frame
y_final = X_test @ w_closed

Id = (df_test.Id).to_numpy()
arr = np.array([Id,y_final]).T
results = pd.DataFrame(data=arr, columns = ['Id','y'])
results.to_csv("results_closed.csv")

In [95]:
w_closed

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

## gradient decent

In [96]:
n = y.shape[0]
print(n)
L = lambda w: np.linalg.norm(y-X@w)**2/n

10000


In [97]:

#gradient decent liner regression
def gradient_descent(X, w_init, learning_rate=0.5, n_steps=1000, tol=1e-5, normalize=True):
    w_curr = np.copy(w_init)
    for step in tqdm(range(n_steps)):
        direction = nd.Gradient(L)((w_curr))#gradiant from L with respect to w?
        if normalize:
            direction /= np.linalg.norm(direction)
        w_next = w_curr - learning_rate*direction
        if np.linalg.norm(direction) < tol:
            w_curr = w_next
            break
        w_curr = w_next
    return w_curr

#gradient decent liner regression
def momentum_gradient_descent(X, w_init, beta = 0.9, learning_rate=0.01, n_steps=1000, tol=1e-5, normalize=True):
    w_curr = np.copy(w_init)
    grad=nd.Gradient(L)
    z_curr = grad((w_init))
    for step in tqdm(range(n_steps)):
        direction = grad((w_curr))#gradiant from L with respect to w?
        if normalize:
            direction /= np.linalg.norm(direction)
        z_next = beta*z_curr + direction 
        w_next = w_curr - learning_rate*z_next
        if np.linalg.norm(direction) < tol:
            w_curr = w_next
            print("we have reached the break point")
            break
        w_curr = w_next
        z_curr = z_next
    return w_curr

In [99]:
w_init = [0,0,0,0,0,0,0,0,0,0]
w_decent = gradient_descent(X,w_init,learning_rate=0.01,n_steps=1000)
#computes the root mean square error
RMSE = mean_squared_error(y, X @ w_decent)**0.5
RMSE

100%|███████████████████████████████████████| 1000/1000 [00:13<00:00, 75.29it/s]


3.64462805004913

In [100]:
w_decent

array([0.10115255, 0.10097507, 0.10066411, 0.10008634, 0.10039348,
       0.09813641, 0.10215232, 0.10026948, 0.09874295, 0.10019087])

In [101]:
# strore return format as Id, y in a data frame
y_final_decent = X_test @ w_decent
Id = (df_test.Id).to_numpy()

arr_d = np.array([Id,y_final_decent]).T
results = pd.DataFrame(data=arr_d, columns = ['Id','y'])
#results.to_csv("results_decent.csv")

## sklearn linear regression test

In [102]:
reg = LinearRegression().fit(X, y)
w_sk = reg.coef_
mean_squared_error(y, X@w_sk)**0.5
reg.score(X, y)

1.0

In [105]:
reg_ridge = Ridge(alpha = 10).fit(X, y)
w_sk = reg_ridge.coef_
print(mean_squared_error(y, X@w_sk)**0.5)
reg_ridge.score(X,y)

3.222330420673134e-07


1.0